In [ ]:
!pip install transformers[all]

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 8.2 MB/s 


In [ ]:
!pip install transformers[sentencepiece]


In [11]:
ch_text = 'hello 我叫萨拉，我住在伦敦。'

## Code


In [ ]:
!pip install python-magic

In [ ]:
!sudo apt-get install libmagic1

In [ ]:
import subprocess as sub

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
#import magic
#import shlex
import os
import re
import pickle
import fileinput
import sys

In [8]:
zh2en_model = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-zh-en')
zh2en_tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-zh-en')

saved_model = pickle.dumps(zh2en_model)
saved_tokenizer = pickle.dumps(zh2en_tokenizer)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/786k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/788k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

In [4]:
def translate_text(ch_text):
  """
  pass your chinese text and it returns its english translation 
  """

  zh2en_model = pickle.loads(saved_model)
  zh2en_tokenizer = pickle.loads(saved_tokenizer)
  zh2en_translation = pipeline('translation_zh_to_en', 
                      model=zh2en_model, 
                      tokenizer=zh2en_tokenizer)
  return zh2en_translation(ch_text)

In [3]:
def get_type_of_file(data_path):
  """
  pass file path with its .format otherwise it faces some problems
  """
  try:
    #return magic.Magic().from_file(data_path)   
    #
    # p = sub.Popen(data_path , stdout=sub.PIPE, stderr=sub.PIPE)
    # output, errors = p.communicate()
    #
    # filename = data_path
    # cmd = shlex.split('file --mime-type {0}'.format(filename))
    # result = sub.check_output(cmd)
    # mime_type = result.split()[-1]
    # return mime_type
    file_name, file_extension = os.path.splitext(data_path)
    return file_extension
  except IOError:
    return 'No File Found'

In [ ]:
def get_index_of_chinese_text(file):


In [135]:
comment_format_start = {'.html':['<!--'],'.htm':['<!--'],'js':['//','/*'],'.java':['//','/*'],'.py':['#']}
comment_format_end = {'.html':'-->','.htm':'-->','js':'*/','java':'*/'}


In [127]:
def replace_text(line):
  """
  pass line you want to change and the text before to replace new line 
  in new text
  """

  if re.search('[\u4e00-\u9fff]',line) != None:
    #line = line.replace('Hello', 'import')
    #sys.stdout.write(line)

    changes = line.replace(line, translate_text(line)[0]['translation_text'])
    return changes 

  else:
    return line 

In [137]:
def process_text(input_path):
  try:
    f = open(input_path,'r')
    file_name, file_type = os.path.splitext(input_path)
    file_type = file_type[1:].lower()
    comment_closed = True
    replacement = []
    first_line =True
    #try:
    if comment_format_start.get(file_type) != None:
      comment_format_s = comment_format_start[file_type]
      #print(comment_format_s)

      for line in f.readlines():
        #print(line)
        if comment_closed:
          #if comment_format_s[0]
          for i in comment_format_s:
            if line.strip().startswith(i):
              #line = translate_text(line)
              replacement.append("{}".format(i) + replace_text(line.split(i)[1]))
              #print('startswith{}'.format(i))
              #try:

              if comment_format_start.get(file_type) != None :
                comment_format_e = comment_format_end[file_type]
                #print('endswith{}'.format(comment_format_e))

                if not line.endswith(comment_format_e) and len(comment_format_s)>1:
                  if i == comment_format_s[1]:
                    comment_closed = False
              #except NameError:
              else:
                print('one line comment')

            if len(line.strip().split(i))>1 and line.strip().split(i)[0] != "" :
              replacement.append(line.strip().split(i)[0]+"{}".format(i)+replace_text(line.strip().split(i)[1]))
              #print('contains one line comment')


            #else:
          if first_line:
            replacement.append(line)
          if len(replacement)>0:
            if replacement[-1] != line[:line.index('\n')]:
              replacement.append(line)
              #print('no comment found')

        else:
          replacement.append(replace_text(line))
          #print('comment continues')

          if line.endswith(comment_format_e):
            comment_closed = True
        first_line = False
    #except NameError:
    else:
      #print('This file doesnt contain comment')
      for line in f.readlines():
        replacement.append(replace_text(line))
    f.close()

    if len(replacement) != 0:
      fout = open("{}_translated.{}".format(file_name,file_type), "w")
      fout.writelines(replacement)
      fout.close()  

  except IOError:
    print('No File Found')



In [138]:
process_text('ResponseCode.java')